In [1]:
import requests
import json
import boto3
from botocore.exceptions import NoCredentialsError
import sys
import re


In [5]:

# AI Model Settings
INFERENCE_ENDPOINT = "https://granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MODEL_API_URL = f"{INFERENCE_ENDPOINT}/v1/completions"
MODEL_NAME = "granite"

# MinIO Storage Settings
MINIO_ENDPOINT = "minio-api-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MINIO_ACCESS_KEY = "minio"
MINIO_SECRET_KEY = "minio123"
MINIO_BUCKET = "logs"
LOG_FILE = "service_error_wentan.txt"

In [9]:

def read_log_from_minio(bucket_name, object_name):
    """Reads a log file's content from a MinIO bucket."""
    print(f"--> Step 1: Reading '{object_name}' from bucket '{bucket_name}'...")
    try:
        s3_client = boto3.client(
            's3',
            endpoint_url=f"http://{MINIO_ENDPOINT}",
            aws_access_key_id=MINIO_ACCESS_KEY,
            aws_secret_access_key=MINIO_SECRET_KEY
        )
        response = s3_client.get_object(Bucket=bucket_name, Key=object_name)
        content = response['Body'].read().decode('utf-8')
        print(f"    ✅ Successfully read log content from MinIO.")
        return content
    except Exception as e:
        print(f"    ❌ An error occurred while reading from MinIO: {e}")
    return None


In [10]:
read_log_from_minio(MINIO_BUCKET, LOG_FILE)

--> Step 1: Reading 'service_error_wentan.txt' from bucket 'logs'...
    ✅ Successfully read log content from MinIO.


'WEBSERVER: UP\n\nTASK [Generate health report] **************************************************  12:51:58\nok: [aiops.demo.vm] => {\n   "msg": "HTTP SERVER HEALTH REPORT\\n=========================\\n[Basic Information]\\n- Hostname: aiops\\n- OS: RedHat 9.5\\n\\n[Web Server Status]\\n- Installed: Yes\\n- Service Status: UNKNOWN\\n- Listening on Port 80: Yes\\n\\n[Security Configuration]\\n\\n\\n[Connectivity]\\n- HTTP Response: 200\\n- Config Valid: Valid\\n"\n}\n\nWEBSERVER: DOWN\n\nTASK [Generate health report] **************************************************  13:15:03\nok: [aiops.demo.vm] => {\n   "msg": "HTTP SERVER HEALTH REPORT\\n=========================\\n[Basic Information]\\n- Hostname: aiops\\n- OS: RedHat 9.5\\n\\n[Web Server Status]\\n- Installed: Yes\\n- Service Status: UNKNOWN\\n- Listening on Port 80: No\\n\\n[Security Configuration]\\n\\n\\n[Connectivity]\\n- HTTP Response: -1\\n- Config Valid: Valid\\n"\n}\n'

In [11]:

def query_model_for_health_report(logs):
    """Queries the AI model to generate a clean health report from Ansible output."""
    print(f"\n--> Step 2: Querying AI model for health report...")
    
    # This new prompt is specifically designed to parse the Ansible output
    # and generate the desired clean report format.
    prompt = f"""
Analyze the following Ansible health check output. The log contains a "WEBSERVER: UP" and a "WEBSERVER: DOWN" state. 
Your task is to synthesize this information and generate a single, clean "HTTP SERVER HEALTH REPORT" for an ideal, healthy server. 
The report must match this exact format, including spacing and categories:

HTTP SERVER HEALTH REPORT
=========================
[Basic Information]
- Hostname: <hostname>
- OS: <Operating System>

[Web Server Status]
- Installed: Yes
- Service Status: RUNNING
- Listening on Port 80: Yes

[Security Configuration]
- Firewall Rules:
    - Allow HTTP (80/tcp)
    - Allow HTTPS (443/tcp)

[Connectivity]
- HTTP Response: 200
- Config Valid: Valid

--- LOGS START ---
{logs}
--- LOGS END ---
"""
    headers = {"Content-Type": "application/json"}
    payload = {"model": MODEL_NAME, "prompt": prompt, "max_tokens": 512}
    
    try:
        response = requests.post(MODEL_API_URL, headers=headers, json=payload, verify=False)
        response.raise_for_status()
        result = response.json()
        if 'choices' in result and result['choices']:
            report_text = result['choices'][0].get('text', 'Error: Could not extract report.')
            print("    ✅ AI analysis complete.")
            return report_text
    except Exception as e:
        print(f"    ❌ Error querying model: {e}")
    return None

# --- Main Execution ---
def main():
    """Main function to run the AIOps workflow."""
    # 1. Read the specified log file from MinIO
    log_content = read_log_from_minio(MINIO_BUCKET, LOG_FILE)
    if not log_content:
        sys.exit("Pipeline stopped: Could not retrieve logs.")
        
    # 2. Query the model to generate the health report
    health_report = query_model_for_health_report(log_content)
    if not health_report:
        sys.exit("Pipeline stopped: Could not generate AI report.")

    # 3. Print the final, clean report
    print("\n" + "="*50)
    print("      AIOps Generated Health Report")
    print("="*50 + "\n")
    print(health_report)


if __name__ == "__main__":
    print("==============================================")
    print("      AIOps PIPELINE: LOG TO HEALTH REPORT")
    print("==============================================")
    main()
    print("\n==============================================")
    print("            PIPELINE COMPLETE")
    print("==============================================")

      AIOps PIPELINE: LOG TO HEALTH REPORT
--> Step 1: Reading 'service_error_wentan.txt' from bucket 'logs'...
    ✅ Successfully read log content from MinIO.

--> Step 2: Querying AI model for health report...


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


    ✅ AI analysis complete.

      AIOps Generated Health Report


HTTP SERVER HEALTH REPORT
[Basic Information]
- Hostname: aiops
- OS: RedHat 9.5

[Web Server Status]
- Installed: Yes
- Service Status: RUNNING
- Listening on Port 80: Yes

[Security Configuration]
- Firewall Rules:
    - Allow HTTP (80/tcp)
    - Allow HTTPS (443/tcp)

[Connectivity]
- HTTP Response: 200
- Config Valid: Valid

Note: The "WEBSERVER: UP" and "WEBSERVER: DOWN" states indicate that the web server is not always running and may experience intermittent issues. It is recommended to investigate and resolve these issues to ensure optimal performance and stability.

HTTP SERVER HEALTH REPORT
[Basic Information]
- Hostname: aiops
- OS: RedHat 9.5

[Web Server Status]
- Installed: Yes
- Service Status: RUNNING
- Listening on Port 80: Yes

[Security Configuration]
- Firewall Rules:
    - Allow HTTP (80/tcp)
    - Allow HTTPS (443/tcp)

[Connectivity]
- HTTP Response: 200
- Config Valid: Valid

Note: The "WEBSERVER: